In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load EEG data from EDF
edf_file = "PN00/PN00-1.edf"  # Replace with your actual file path
raw = mne.io.read_raw_edf(edf_file, preload=True)

Extracting EDF parameters from c:\Users\Shuvadipta Biswas\Desktop\DEMO_CAPSTONE\PN00\PN00-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\Shuvadipta Biswas\AppData\Local\Temp\ipykernel_13648\2441966496.py:3: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)
C:\Users\Shuvadipta Biswas\AppData\Local\Temp\ipykernel_13648\2441966496.py:3: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)
C:\Users\Shuvadipta Biswas\AppData\Local\Temp\ipykernel_13648\2441966496.py:3: RuntimeWarning: Highpass cutoff frequency 15.91549 is greater than lowpass cutoff frequency 15.0, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


Reading 0 ... 1343999  =      0.000 ...  2624.998 secs...


In [3]:
# Select 19 standard EEG channels
selected_channels = [
    "EEG Fp1", "EEG Fp2", "EEG F3", "EEG F4", "EEG C3", "EEG C4", "EEG P3", "EEG P4", "EEG O1", "EEG O2",
    "EEG F7", "EEG F8", "EEG T3", "EEG T4", "EEG T5", "EEG T6", "EEG Fz", "EEG Cz", "EEG Pz"
]
raw.pick_channels(selected_channels)

# Remove "EEG " prefix
new_channel_names = {ch: ch.replace("EEG ", "") for ch in raw.ch_names}
raw.rename_channels(new_channel_names)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawEDF | PN00-1.edf, 19 x 1344000 (2625.0 s), ~194.8 MiB, data loaded>

In [4]:
# Apply band-pass filter (1-40 Hz for EEG analysis)
raw.filter(1, 40, fir_design='firwin')

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (3.303 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.6s finished


<RawEDF | PN00-1.edf, 19 x 1344000 (2625.0 s), ~194.8 MiB, data loaded>

In [5]:
import pywt

In [6]:
def denoise_signal(signal, wavelet='db1', level=1):
    # Perform Discrete Wavelet Transform (DWT)
    coeffs = pywt.wavedec(signal, wavelet, level=level)

    # Set threshold value for coefficients
    threshold = 0.2 * np.max(np.abs(coeffs[-level]))

    # Threshold the coefficients
    coeffs = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]

    # Perform Inverse Discrete Wavelet Transform (IDWT)
    denoised_signal = pywt.waverec(coeffs, wavelet)

    return denoised_signal

In [7]:
denoised_data = np.array([denoise_signal(raw.get_data(picks=ch)[0]) for ch in range(len(raw.info['ch_names']))])

In [8]:
raw_denoised = raw.copy()
raw_denoised._data = denoised_data

In [9]:
raw = raw_denoised

In [10]:
# Select channels (assumes a standard EEG montage)
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)

<RawEDF | PN00-1.edf, 19 x 1344000 (2625.0 s), ~194.9 MiB, data loaded>

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os

In [12]:
# Parameters
seizure_start = 1143  # in seconds
sampling_rate = raw.info['sfreq']
output_dir = 'PN00_beforeSZ'  # Output directory

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Loop through every 5-second window from seizure_start - 120 to seizure_start
for sec in range(seizure_start - 120, seizure_start, 5):
    # Get data from the current 5-second window
    start_sample = int(sec * sampling_rate)
    stop_sample = int((sec + 5) * sampling_rate)
    data, times = raw[:, start_sample:stop_sample]

    # Compute alpha band power (8–12 Hz) using Welch's method
    psds, freqs = mne.time_frequency.psd_array_welch(
        data, sfreq=sampling_rate, fmin=8, fmax=13, n_fft=256
    )
    psd_alpha = np.mean(psds, axis=1)

    # Plot topomap
    fig, ax = plt.subplots(figsize=(8, 6))
    mne.viz.plot_topomap(psd_alpha, raw.info, cmap='viridis', axes=ax, show=False)

    # Save figure with window label
    fig.savefig(f'{output_dir}/img{sec}_to_{sec+5}.png', dpi=300)
    plt.close(fig)

Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [13]:
# Parameters
seizure_end = 1213  # in seconds
sampling_rate = raw.info['sfreq']
output_dir = 'PN00_afterSZ'  # Output directory

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Loop through every 5-second window from 1213 to 1333
for sec in range(seizure_end, seizure_end + 120, 5):
    # Get data from the current 5-second window
    start_sample = int(sec * sampling_rate)
    stop_sample = int((sec + 5) * sampling_rate)
    data, times = raw[:, start_sample:stop_sample]

    # Compute alpha band power (8–12 Hz) using Welch's method
    psds, freqs = mne.time_frequency.psd_array_welch(
        data, sfreq=sampling_rate, fmin=8, fmax=13, n_fft=256
    )
    psd_alpha = np.mean(psds, axis=1)

    # Plot topomap
    fig, ax = plt.subplots(figsize=(8, 6))
    mne.viz.plot_topomap(psd_alpha, raw.info, cmap='viridis', axes=ax, show=False)

    # Save figure with window label
    fig.savefig(f'{output_dir}/img{sec}_to_{sec+5}.png', dpi=300)
    plt.close(fig)

Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [14]:
# Parameters
seizure_start = 1143  # in seconds
sampling_rate = raw.info['sfreq']
output_dir = 'PN00_duringSZ'  # Output directory

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Loop through every 5-second window from 1143 to 1213
for sec in range(seizure_start, seizure_start + 70, 5):
    # Get data from the current 5-second window
    start_sample = int(sec * sampling_rate)
    stop_sample = int((sec + 5) * sampling_rate)
    data, times = raw[:, start_sample:stop_sample]

    # Compute alpha band power (8–12 Hz) using Welch's method
    psds, freqs = mne.time_frequency.psd_array_welch(
        data, sfreq=sampling_rate, fmin=8, fmax=13, n_fft=256
    )
    psd_alpha = np.mean(psds, axis=1)

    # Plot topomap
    fig, ax = plt.subplots(figsize=(8, 6))
    mne.viz.plot_topomap(psd_alpha, raw.info, cmap='viridis', axes=ax, show=False)

    # Save figure with window label
    fig.savefig(f'{output_dir}/img{sec}_to_{sec+5}.png', dpi=300)
    plt.close(fig)

Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Effective window size : 0.500 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
